In [1]:
import os
import sys
import pandas as pd
import json
import numpy as np

os.chdir(sys.path[0])
f=open(r"analysis_input.json")
input=json.load(f)
f.close()
f=open(r"input.json")
settings=json.load(f)
f.close()
df=pd.read_csv("Desired.csv", index_col=0, na_values='')

In [2]:
sample_size=settings["sample_size"]

In [3]:

class keep:
    all=[]
    def __init__(self,name, before='', after=''):
        self.name=name
        self.before=before
        self.after=after
        self.df=pd.DataFrame({})
        keep.all.append(self)
    def init_df(self,df,column_header,number):
        self.df[self.name+' '+str(number)]=df[column_header].apply(lambda x:self.get_value(x))
        if len(df.keys())>3:
            self.summary=self.df.apply(keep.top3_edit, axis=1) ## maybe i use this only for hexa
        else:
            self.summary=self.df.sum(axis=1)
    def inside(self, string):
        if string.endswith(self.after) and string.startswith(self.before):
            return True
        else:
            return False
    def get_value(self, string):
        if self.inside(string):
            x=string.replace(self.before,"",1)
            x=x.removesuffix(self.after)
            return int(x)
        else: return 0 # maybe changing to  return none
    def top3_edit(row):
        temp_list=row[row.keys()].values.tolist()   
        temp_list= list(filter(lambda num:num>0,temp_list))
        if len(temp_list)>3:
            temp_list.sort(reverse=True)
            return sum(temp_list[0:3])
        else:
            return sum(temp_list)
    def inside_all(string):
        for x in keep.all:
            if x.inside(string):
                return True
        return False
before=0
after=1
keep_dict={}
for x in input.keys():
    keep_dict[x]=keep(x,input[x][before],input[x][after])
del before, after, input
#theres 2 collections keep_dict and keep.list


In [4]:
header=df.keys()
header=list(filter(lambda x: x.startswith("Line "),header))
header

['Line 1', 'Line 2', 'Line 3', 'Line 4', 'Line 5', 'Line 6']

In [5]:
count=0
for x in header:
    count+=1
    df[x]=df[x].apply(lambda x: x if keep.inside_all(x) else '')
    for y in keep.all:
        y.init_df(df,x,count)
for x in keep.all:
    df[x.name]=x.summary


In [6]:
df.to_csv("analysed.csv")

In [7]:
df

,Line 1,Line 2,Line 3,Line 4,Line 5,Line 6,desired,Attack %,IED %
4,,,Ignore Enemy Defense +30%,ATT +9%,,,2,9,30
15,,ATT +9%,,ATT +9%,,,2,18,0
30,,,Ignore Enemy Defense +30%,,,Ignore Enemy Defense +30%,2,0,60
37,,Ignore Enemy Defense +30%,,,ATT +9%,,2,9,30
38,,ATT +9%,,ATT +9%,Ignore Enemy Defense +30%,,3,18,30
...,...,...,...,...,...,...,...,...,...
99954,ATT +12%,,,,,ATT +9%,2,21,0
99960,Ignore Enemy Defense +40%,ATT +9%,,,,,2,9,40
99965,,ATT +9%,,,Ignore Enemy Defense +30%,,2,9,30
99979,,,ATT +9%,ATT +9%,,,2,18,0


In [8]:
#import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [13]:
print(keep.all[0].name)
print(keep.all[0].before)
print(keep.all[0].after)

Attack %
ATT
%


In [29]:
def generate_graph(keep_object):
    new_df=df[keep_object.name]
    x_axis = list(set(new_df.unique()))
    x_axis.sort()
    y_axis=[0]
    for x in x_axis:
        if x ==0:
            continue
        y_axis.append(new_df[new_df==x].count())
    z_axis=list(np.cumsum(y_axis[::-1]))[::-1]
    x_title=keep_object.name + " with " + str(sample_size) + "samples"
    y_title="Count at "
    z_title="Total"
    z_axis[0]=sample_size
    y_axis[0]=sample_size-sum(y_axis)

    fig = make_subplots(rows=2, cols=1, row_heights=[.85,.15], shared_xaxes=True)

    bar_chart = go.Bar(x=x_axis,y=y_axis, name=y_title, text=y_axis)
    fig.add_trace(bar_chart, row=1,col=1)

    cum_chart = go.Scatter(x=x_axis,y=z_axis, name=z_title)
    fig.append_trace(cum_chart, row=1,col=1)

    box_plot=go.Box(x=new_df, notched=True, name="Boxplot")
    fig.add_trace(box_plot, row=2, col=1)
    fig.update_layout(title_text=x_title)
    fig.show()

In [30]:
generate_graph(keep.all[0])

In [26]:
df2=df["Attack %"]
x_axis = list(set(df2.unique()))
x_axis.sort()
y_axis=[0]
for x in x_axis:
    if x ==0:
        continue
    y_axis.append(df2[df2==x].count())
z_axis=list(np.cumsum(y_axis[::-1]))[::-1]
x_title="Attack %"
y_title="Count at "
z_title="Total"
z_axis[0]=sample_size
y_axis[0]=sample_size-sum(y_axis)

In [ ]:
#total number of cubes
#put a pie chart at the side

In [ ]:
df2

4         9
15       18
30        0
37        9
38       18
         ..
99954    21
99960     9
99965     9
99979    18
99996    12
Name: Attack, Length: 14803, dtype: int64

In [ ]:
x_axis.sort()
print(x_axis)
print(y_axis)
z_axis=list(np.cumsum(y_axis[::-1]))[::-1]
print(z_axis)

[0, 9, 12, 18, 21, 24, 27, 30, 33]
[0, 6211, 1055, 2514, 1076, 39, 150, 144, 8]
[11197, 11197, 4986, 3931, 1417, 341, 302, 152, 8]


In [ ]:
x=[1,2,3,4,5]
def func(x):
    return x+1
x=list(map(func, x))
print(x)


[2, 3, 4, 5, 6]
